In [50]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import tensorflow as tf

In [42]:
file_path = r"D:\Coding_Dev\College\SEM_6\ML\Assets\breast_cancer_wisconsin_original\breast-cancer-wisconsin.data"

def load_breast_cancer_data():
    data = pd.read_csv(file_path)
    columns = ["Feature0","Feature1", "Feature2", "Feature3", "Feature4", "Feature5", "Feature6", "Feature7", "Feature8", "Feature9", "Class"]
    data = pd.read_csv(file_path, header=None, sep=",", names=columns)
    data.drop("Feature0", axis=1, inplace=True)
    data.replace('?', pd.NA, inplace=True)
    data.dropna(inplace=True)
    data['Feature6'] = data['Feature6'].astype('int64')
    
    X = data.iloc[:, :-1].values
    y = data.iloc[:, -1].values
    
    
    X = (X - np.mean(X, axis=0)) / np.std(X, axis=0)
    return X, y

X, y = load_breast_cancer_data()


In [43]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [44]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))
def sigmoid_derivative(x):
    return x * (1 - x)

In [45]:
def train_mlp(X_train, y_train, hidden_layer_size, learning_rate, epochs):
    # Initialize weights for input and hidden layers
    input_size = X_train.shape[1]
    output_size = 1  # Binary classification

    np.random.seed(0)
    weights_input_hidden = np.random.rand(input_size, hidden_layer_size)
    weights_hidden_output = np.random.rand(hidden_layer_size, output_size)

    for epoch in range(epochs):
        # Forward propagation
        hidden_layer_input = np.dot(X_train, weights_input_hidden)
        hidden_layer_output = sigmoid(hidden_layer_input)

        output_layer_input = np.dot(hidden_layer_output, weights_hidden_output)
        output_layer_output = sigmoid(output_layer_input)

        # Backpropagation
        error = y_train.reshape(-1, 1) - output_layer_output
        d_output = error * sigmoid_derivative(output_layer_output)

        error_hidden_layer = d_output.dot(weights_hidden_output.T)
        d_hidden_layer = error_hidden_layer * sigmoid_derivative(hidden_layer_output)

        # Update weights
        weights_hidden_output += hidden_layer_output.T.dot(d_output) * learning_rate
        weights_input_hidden += X_train.T.dot(d_hidden_layer) * learning_rate


    return weights_input_hidden, weights_hidden_output


In [76]:
import numpy as np
import pandas as pd

file_path = r"D:\Coding_Dev\College\SEM_6\ML\Assets\breast_cancer_wisconsin_original\breastCancer.csv"
data = pd.read_csv(file_path)

data = data.select_dtypes(include=[np.number])


X = data.drop('class', axis=1).values


y_labels = data['class'].values
y_unique = np.unique(y_labels)
y = np.zeros((len(y_labels), len(y_unique)))
for i, label in enumerate(y_labels):
    y[i, np.where(y_unique == label)] = 1


X = (X - X.mean(axis=0)) / X.std(axis=0)


def sigmoid(x):
    return 1 / (1 + np.exp(-x))
def sigmoid_derivative(x):
    return x * (1 - x)
def mean_squared_error(y_true, y_pred):
    return np.mean(np.sum((y_true - y_pred) ** 2, axis=1))


input_size = X.shape[1]
output_size = y.shape[1]


hidden_layer_sizes = [2, 4, 8, 16, 32]


learning_rate = 0.01
epochs = 1000

error_rates = []


for hidden_size in hidden_layer_sizes:
    np.random.seed(0)
    weights_input_hidden = np.random.uniform(size=(input_size, hidden_size))
    biases_hidden = np.zeros((1, hidden_size))
    weights_hidden_output = np.random.uniform(size=(hidden_size, output_size))
    biases_output = np.zeros((1, output_size))
    
    
    for epoch in range(epochs):
        
        hidden_layer_input = np.dot(X, weights_input_hidden) + biases_hidden
        hidden_layer_output = sigmoid(hidden_layer_input)
        output_layer_input = np.dot(hidden_layer_output, weights_hidden_output) + biases_output
        output_layer_output = sigmoid(output_layer_input)
        
        
        loss = mean_squared_error(y, output_layer_output)
        
        
        dloss_doutput = 2 * (output_layer_output - y)
        doutput_doutput_input = sigmoid_derivative(output_layer_output)
        dloss_doutput_input = dloss_doutput * doutput_doutput_input
        
        dloss_dhidden_output = dloss_doutput_input.dot(weights_hidden_output.T)
        dhidden_output_dhidden_input = sigmoid_derivative(hidden_layer_output)
        dloss_dhidden_input = dloss_dhidden_output * dhidden_output_dhidden_input
        
        
        weights_hidden_output -= hidden_layer_output.T.dot(dloss_doutput_input) * learning_rate
        biases_output -= np.sum(dloss_doutput_input, axis=0, keepdims=True) * learning_rate
        weights_input_hidden -= X.T.dot(dloss_dhidden_input) * learning_rate
        biases_hidden -= np.sum(dloss_dhidden_input, axis=0, keepdims=True) * learning_rate
    
    
    predictions = (output_layer_output > 0.5).astype(int)
    accuracy = np.mean(np.all(predictions == y, axis=1))
    error_rate = 1 - accuracy
    error_rates.append((hidden_size, error_rate))


for hidden_size, error_rate in error_rates:
    print(f'Hidden Layer Size: {hidden_size}, Accuracy: {1-error_rate}')

Hidden Layer Size: 2, Accuracy: 0.9785407725321889
Hidden Layer Size: 4, Accuracy: 0.977110157367668
Hidden Layer Size: 8, Accuracy: 0.9756795422031473
Hidden Layer Size: 16, Accuracy: 0.6366237482117311
Hidden Layer Size: 32, Accuracy: 0.6366237482117311
